<a href="https://colab.research.google.com/github/SandeepKonduruFeb12/aiml/blob/master/silver/A4Redis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Set up and demonstrate an LLM application using LangChain, integrated with Upstash Redis for data storage and retrieval, including steps for creating a Redis instance, uploading sample data, and querying it via the LLM.

## Create Upstash Redis Instance

### Subtask:
Guide the user to sign up for Upstash, create a new Redis database, and obtain the connection URL and token for the instance.


### Subtask:
Guide the user to sign up for Upstash, create a new Redis database, and obtain the connection URL and token for the instance.

#### Instructions
1. **Navigate to the Upstash website (https://upstash.com/)** and sign up for a free account if you don't already have one.
2. Once logged in, **create a new Redis database**. Follow the on-screen prompts to configure your database (e.g., select a region).
3. After the database is created, **locate the connection details** for your new Redis instance. You will need the `UPSTASH_REDIS_REST_URL` and `UPSTASH_REDIS_REST_TOKEN`.

## Install Required Libraries

### Subtask:
Install the necessary Python libraries: `redis` for connecting to Upstash, `langchain` for the LLM framework, and the client library for your chosen LLM (e.g., `openai`, `google-generativeai`).


In [1]:
print("Installing required libraries: redis, langchain, openai, ...")

# Pip will now automatically determine compatible versions based on langchain and langchain-openai.
!pip install --upgrade langchain langchain-core langchain-openai redis openai langchain-community

print("\nVerifying installed versions of langchain and langchain-community...")
!pip show langchain
!pip show langchain-community


Installing required libraries: redis, langchain, openai, ...

Verifying installed versions of langchain and langchain-community...
Name: langchain
Version: 1.1.0
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Name: langchain-community
Version: 0.4.1
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain-classic, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from google.colab import userdata

# --- 1. Initialize the LLM ---
# Get the corporate API key
hcl_api_key = userdata.get('hcl')

# Ensure the API key is also set as an environment variable for the OpenAI client
os.environ["OPENAI_API_KEY"] = hcl_api_key # This is picked up by the underlying openai client

# --- Endpoint config ---
API_URL = (
    "https://aicafe.hcl.com/AICafeService/api/v1/subscription/openai/"
    "deployments/gpt-4.1/chat/completions?api-version=2024-12-01-preview"
)

llm = ChatOpenAI(
    base_url=API_URL,
    api_key=hcl_api_key, # Pass API_KEY here to satisfy the internal client requirement
    # Removed model parameter as it's often redundant with base_url for custom endpoints
    temperature=0,
    extra_headers={"api-key": hcl_api_key} # Explicitly pass API key in 'api-key' header
) # I want to minimize hallucination - temperature = 0 makes the model output more deterministic


# --- 2. Configure Redis-backed message history ---

UPSTASH_REDIS_REST_URL = userdata.get('upstash_url')
UPSTASH_REDIS_REST_TOKEN = userdata.get('upstash_token')
# Function to get session history
def get_session_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(
        url=UPSTASH_REDIS_REST_URL,
        session_id=session_id
    )

print("Configured function to get RedisChatMessageHistory.")

# --- 3. Create a LangChain LCEL chain with RunnableWithMessageHistory ---
# Define a chat prompt template that includes a MessagesPlaceholder for history
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a friendly AI assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# Create the LCEL chain
chain = prompt | llm
print("Created LCEL chain (prompt | llm).")

# Wrap the chain with RunnableWithMessageHistory
# The input_messages_key should match the variable_name in MessagesPlaceholder
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)
print("Wrapped chain with RunnableWithMessageHistory for persistent chat history.")

# --- Demonstrate conversational context ---
config = {"configurable": {"session_id": "my-lcel-session"}}

# Clear existing Redis history for this session to avoid validation errors from old data
print("Clearing existing Redis chat history for 'my-lcel-session'...")
get_session_history("my-lcel-session").clear()
print("Redis chat history cleared.")

print("\n--- First conversation turn --- ")
response1 = with_message_history.invoke({"question": "Hi, my name is Alice. What can you do?"}, config=config)
print(f"AI Response: {response1.content}")

print("\n--- Second conversation turn (should remember name) --- ")
response2 = with_message_history.invoke({"question": "What is my name?"}, config=config)
print(f"AI Response: {response2.content}")

print("\n--- Third conversation turn --- ")
response3 = with_message_history.invoke({"question": "Can you tell me more about Large Language Models?"}, config=config)
print(f"AI Response: {response3.content}")


# You can also inspect the Redis memory directly for the session
print("\n--- Inspecting Redis chat history for 'my-lcel-session' ---")
redis_chat_history = get_session_history("my-lcel-session").messages
for msg in redis_chat_history:
    print(f"  {msg.type}: {msg.content}")

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  if (await self.run_code(code, result,  async_=asy)):


Configured function to get RedisChatMessageHistory.
Created LCEL chain (prompt | llm).
Wrapped chain with RunnableWithMessageHistory for persistent chat history.
Clearing existing Redis chat history for 'my-lcel-session'...
Redis chat history cleared.

--- First conversation turn --- 
AI Response: Hi Alice! It’s great to meet you. I can help with a wide range of tasks, such as:

- Answering questions on many topics (science, history, technology, etc.)
- Helping you write emails, essays, or creative stories
- Summarizing articles or documents
- Providing recommendations (books, movies, recipes, etc.)
- Assisting with math problems or coding
- Organizing schedules or making to-do lists
- Translating text between languages

If you have something specific in mind, just let me know!

--- Second conversation turn (should remember name) --- 
AI Response: Your name is Alice!

--- Third conversation turn --- 
AI Response: Absolutely, Alice! Large Language Models (LLMs) are a type of artificial 